# Example: Parallel Network using LFPy

This example notebook submits a job to a chosen compute cluster (**JUSUF**), loads a **Singularity** container with a working **LFPy** installation and executes a version of the LFPy example file ``example_network.py`` (https://github.com/LFPy/LFPy/blob/master/examples/example_network/example_network.py) in parallel on one HPC backend of EBRAINS/HBP.

The example demonstrates usage of ``LFPy.Network`` with network of ball-and-stick type
morphologies with active HH channels inserted in the somas and passive-leak
channels distributed throughout the apical dendrite. The corresponding
morphology and template specifications are in the files ``BallAndStick.hoc`` and
``BallAndStickTemplate.hoc``.


For more indepth info on using HPC backends via EBRAINS see https://wiki.ebrains.eu/bin/view/Collabs/using-supercomputers-from-the-collab/

## Prepare job

In [ ]:
# use the pyunicore library
!pip install pyunicore --upgrade

In [ ]:
# import modules
import os
import pyunicore.client as unicore_client
import requests
import json
from time import sleep, time
from pprint import pprint
from IPython.display import IFrame

In [ ]:
# Create connection to supercomputer (i.e., JUSUF)
tr = unicore_client.Transport(clb_oauth.get_token())
r = unicore_client.Registry(tr, unicore_client._HBP_REGISTRY_URL)

In [ ]:
# Valid choices for supercomputers are one of the keys in:
r.site_urls

In [ ]:
# connect to HPC resouce - change subpercomputer name to whatever you have access to:
supercomputer = 'JUSUF'
try:
    site_client = r.site(supercomputer)
except KeyError:
    # cluster may be dropped from Registry.site_urls for whatever reason
    site_client = unicore_client.Client(r.transport, 'https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core')

In [ ]:
# check connection to supercomputer
headers = {}
headers["Authorization"] = "Bearer " + clb_oauth.get_token()
headers['Accept'] = "application/json"
rs = requests.get(url=site_client.site_url, headers = headers, verify = False)
print("Status code %s " % rs.status_code)
print("Content-type %s " % rs.headers['Content-Type'])
reply = rs.json()
# print(json.dumps(reply, indent = 1))

## Download LFPy example network files
Prepare simulation files using example files from the main LFPy repository (https://github.com/LFPy/LFPy.git)

In [ ]:
!wget -O example_network.py https://raw.githubusercontent.com/LFPy/LFPy/master/examples/example_network/example_network.py
!wget -O BallAndStick.hoc https://raw.githubusercontent.com/LFPy/LFPy/master/examples/example_network/BallAndStick.hoc
!wget -O BallAndStickTemplate.hoc https://raw.githubusercontent.com/LFPy/LFPy/master/examples/example_network/BallAndStickTemplate.hoc

In [ ]:
## to make things more interesting with HPCs, let's increase the network size and adjust synaptic weights by applying a patch:
diff = '''240c240
< population_sizes = [80, 20]
---
> population_sizes = [2048, 512]
257,260c257,260
< weightArguments = [[dict(loc=0.002, scale=0.0002),
<                     dict(loc=0.002, scale=0.0002)],
<                    [dict(loc=0.02, scale=0.002),
<                     dict(loc=0.02, scale=0.002)]]
---
> weightArguments = [[dict(loc=0.0005, scale=0.0001),
>                     dict(loc=0.0005, scale=0.0001)],
>                    [dict(loc=0.005, scale=0.001),
>                     dict(loc=0.005, scale=0.001)]]
380c380
<             ax.plot(t[t >= 200], g[t >= 200], '|', label=name)
---
>             ax.plot(t[t >= 200], g[t >= 200], '.', ms=2, label=name)
'''

with open('patch.diff', 'w') as f:
    f.writelines(diff)

In [ ]:
# apply patch
!patch example_network.py patch.diff

## Prepare singularity container
See https://gitlab.version.fz-juelich.de/bvonstvieth_publications/container_userdoc_tmp for details. 

This step builds the singularity container. It is optional if the recipe has already been uploaded and built on the system. 

The procedure may be different on different HPC backends. 
The container can either way be built from the same recipe: https://raw.githubusercontent.com/LFPy/LFPy/2.2.dev0/Dockerfile

Turn the below Raw block into Code in order to run:

## Prepare main simulation job
This step combines in a single session the following:

- download LFPy container
- upload simulation files from the collab
- ask for resources (# nodes, # MPI processes, # seconds runtime)
- execute simulation
- zip simulation output

In [ ]:
# create dictionary with job specification and define list of input files from this Collab
# simulation_job = {"Job type": "interactive"}
simulation_job = {}
simulation_inputs = ["example_network.py", "BallAndStick.hoc", "BallAndStickTemplate.hoc"] 

In [ ]:
# Resources
simulation_job['Resources'] = {
    "Queue": "batch",
    #"CPUs": "256",
    "Nodes": "2",
    "CPUsPerNode": "128",
    "Runtime": "600",
}

In [ ]:
# commands run on login node before job execution
simulation_job["User precommand"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools
sib download --recipe-name lfpymaster
"""
simulation_job["RunUserPrecommandOnLoginNode"] = "true"

In [ ]:
# - set some environment variables
# - run the python code using interpreter embedded in container
simulation_job["Executable"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools
unset DISPLAY  # matplotlib may look for a nonexistant display on compute node 
srun singularity exec lfpymaster.sif python3 -u example_network.py
"""

In [ ]:
# commands run after job is done
simulation_job["User postcommand"] = "tar -cvf example_network_output.tar example_network_output" 
simulation_job["RunUserPostcommandOnLoginNode"] = "true"

In [ ]:
simulation_job

In [ ]:
# create job
job = site_client.new_job(job_description=simulation_job, inputs=simulation_inputs)

In [ ]:
job.properties

In [ ]:
# wait while job is queing and running
while job.is_running():
    sleep(10)

In [ ]:
job.working_dir.listdir().keys()

In [ ]:
# STDERR output (if any)
stderr = job.working_dir.stat('stderr')
pprint(stderr.raw().readlines()[:5])
pprint(stderr.raw().readlines()[-5:])

In [ ]:
# STDOUT output
stdout = job.working_dir.stat('stdout')
pprint(stdout.raw().readlines()[:10])
pprint(stdout.raw().readlines()[-10:])

In [ ]:
# download simulation output
job.working_dir.stat('example_network_output.tar').download('example_network_output.tar')

In [ ]:
# kill job, clean up files on the remote
job.delete()

## Investigate simulation output

In [ ]:
# untar output to folder example_network_output
!tar -xf example_network_output.tar

In [ ]:
# quick look at e.g., the extracellular potential and spike raster plot
IFrame("./example_network_output/extracellular_potential.pdf", width=800, height=600)

In [ ]:
IFrame("./example_network_output/spike_raster.pdf", width=800, height=600)